# Unit 5 - Groupby
---

1. [Simple groupby](#section1)
2. [Working with dates](#section2)
3. [Groupby on two or more attributes](#section3)
4. [Groupby with a lambda function](#section4)
5. [Groupby with multiple functions](#section5)



##### One of the most useful functions

[groupby documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

#### Split to groups by some criteria + do something with each group seperatly

In [1]:
import pandas as pd
import numpy as np

In [2]:
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv'
vacc_df = pd.read_csv(url)

## 1. Simple groupby

Groupby location:\
Nothing happens here, since we didn't tell indicate what to do with each group\
But: no error. The split is valid :-)

In [3]:
grouped = vacc_df.groupby('location')
grouped

The `median` of `daily_vaccinations` according to `location`:

In [4]:
med_df = vacc_df.groupby('location')[['daily_vaccinations']].median()
med_df

,daily_vaccinations
location,
Afghanistan,9676.0
Africa,905283.0
Albania,1796.5
Algeria,19522.0
Andorra,54.0
...,...
Wallis and Futuna,7.0
World,9371596.0
Yemen,1326.0


Note that this format means `location` is now the index

this means `vacc_df[["location"]]` won't work anymore

##### If you plan to continue using this data and need the index as an attribute:

##### add `reset_index()` and then assign

In [5]:
med_df = med_df.reset_index()
med_df[["location"]]

,location
0,Afghanistan
1,Africa
2,Albania
3,Algeria
4,Andorra
...,...
230,Wallis and Futuna
231,World
232,Yemen
233,Zambia


-----
##### So now we are ready to answer the questions:
##### How do we fill missing values for `total_vaccinations` according to the mean of each country?

We now understand this:

In [6]:
vacc_df.groupby(['location'])[['total_vaccinations']].fillna(method='ffill').reset_index()

,index,total_vaccinations
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
164191,164191,12219760.0
164192,164192,12219760.0
164193,164193,12219760.0
164194,164194,12219760.0


Advanced comment: \
`.mean()` is a built-in **aggregation** function\
`.fillna()` is a built-in **transformation** function\
groupby allows you to aggregte, transform, or filter the data.


### <span style="color:blue"> Exercise:</span>
> What is the average (mean) of the `daily_vaccinations` in each location?
>
> If we do not reset the index, how can we see the `index`?


## 2. Working with dates

How do we extract the month? Currently `date` is an object:

In [7]:
vacc_df[['date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164196 entries, 0 to 164195
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    164196 non-null  object
dtypes: object(1)
memory usage: 1.3+ MB


First, change the `date` into a `datetime` object and extract the month

In [8]:
vacc_df['date'] = pd.to_datetime(vacc_df['date'])
vacc_df[['date']].dtypes

date    datetime64[ns]
dtype: object

In [9]:
vacc_df['month'] = pd.DatetimeIndex(vacc_df['date']).month
vacc_df[['month','date']].head(3)

,month,date
0,2,2021-02-22
1,2,2021-02-23
2,2,2021-02-24


You can use any combination [from here](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

In [10]:
vacc_df['year-month'] = pd.DatetimeIndex(vacc_df['date']).strftime('%Y-%m')
vacc_df[["year-month",'date']]

,year-month,date
0,2021-02,2021-02-22
1,2021-02,2021-02-23
2,2021-02,2021-02-24
3,2021-02,2021-02-25
4,2021-02,2021-02-26
...,...,...
164191,2022-10,2022-10-05
164192,2022-10,2022-10-06
164193,2022-10,2022-10-07
164194,2022-10,2022-10-08


### <span style="color:blue"> Exercise:</span>
> Extract the `year` and add it as a new column called `year` in `vacc_df`
>
> Extract the name of the day and add it as a new column called `weekday` in `vacc_df`
>
> Run the sanity check: `vacc_df[["date","year","weekday"]]` 

In [11]:
# sanity check
# vacc_df[["date","year","weekday"]]

## 3. Groupby on two or more attributes

Now, groupby both `location` and `month`

In [12]:
vacc_df.groupby(['location','month'])[['daily_vaccinations', 'total_vaccinations']].mean().reset_index()

,location,month,daily_vaccinations,total_vaccinations
0,Afghanistan,1,9744.919355,7.352774e+06
1,Afghanistan,2,35161.806452,8.427326e+06
2,Afghanistan,3,26418.129032,1.195944e+07
3,Afghanistan,4,6828.258065,5.835056e+06
4,Afghanistan,5,7195.258065,3.060851e+06
...,...,...,...,...
2792,Zimbabwe,8,31094.935484,7.306646e+06
2793,Zimbabwe,9,20024.150000,6.536451e+06
2794,Zimbabwe,10,15905.975000,6.577630e+06
2795,Zimbabwe,11,22653.966667,6.248622e+06


### <span style="color:blue"> Exercise:</span>
> 
> what will happen if we swith the order of the indexes: `['month', 'location']`?

## 4. Aggregation with a user defined function

Groupby the mean using a lambda function:

In [13]:
vacc_df.groupby(['location', 'month'])[['daily_vaccinations', 'total_vaccinations']].\
agg(lambda x: np.log(x.mean()) if x.mean()!=0 else  0  ).reset_index()

,location,month,daily_vaccinations,total_vaccinations
0,Afghanistan,1,9.184501,15.810588
1,Afghanistan,2,10.467716,15.946990
2,Afghanistan,3,10.181806,16.297032
3,Afghanistan,4,8.828825,15.579395
4,Afghanistan,5,8.881177,14.934204
...,...,...,...,...
2792,Zimbabwe,8,10.344800,15.804295
2793,Zimbabwe,9,9.904694,15.692905
2794,Zimbabwe,10,9.674450,15.699185
2795,Zimbabwe,11,10.028090,15.647872


### <span style="color:blue"> Exercise:</span>
>
> Create your own lambda function that returns 1/x.sum()

## 5. Multiple aggregations

In [14]:
vacc_group = vacc_df.groupby('location').\
agg({'daily_people_vaccinated': ['first', 'last' , 'mean', 'median', 'max'],\
     'total_vaccinations':['max', lambda x: x.max()/1000000]     
    })
vacc_group = vacc_group.reset_index()
vacc_group

location daily_people_vaccinated                          \
                                         first      last          mean   
0          Afghanistan                  1367.0    9991.0  1.884408e+04   
1               Africa                     0.0  343074.0  6.332570e+05   
2              Albania                    64.0      31.0  1.688170e+03   
3              Algeria                    30.0       0.0  1.345704e+04   
4              Andorra                    66.0       0.0  7.545538e+01   
..                 ...                     ...       ...           ...   
230  Wallis and Futuna                   272.0       0.0  9.922504e+00   
231              World                     0.0   10567.0  5.795624e+06   
232              Yemen                  4276.0      67.0  1.491194e+03   
233             Zambia                   106.0    3388.0  1.615831e+04   
234           Zimbabwe                   425.0     332.0  1.076668e+04   

                           total_vaccinations                
        median         max                max    <lambda_0>  
0       7585.0    188998.0       1.658658e+07     16.586584  
1     613776.5   1792254.0       7.995330e+08    799.533040  
2        516.5      6816.0       3.070468e+06      3.070468  
3       9059.0    105248.0       1.526744e+07     15.267442  
4          1.0       854.0       1.569570e+05      0.156957  
..         ...         ...                ...           ...  
230        3.0       272.0       1.805800e+04      0.018058  
231  2647984.0  21071041.0       1.337684e+10  13376.837612  
232      609.0     10240.0       1.258021e+06      1.258021  
233     8387.0     47250.0       1.279211e+07     12.792112  
234     5508.0     85662.0       1.222275e+07     12.222754  

[235 rows x 8 columns]

## 6. Tidy your output



If you want to access the data and not deal with a multi-index, flatten the data by dropping a level and rename the columns:

In [15]:
vacc_group.columns

MultiIndex([(               'location',           ''),
            ('daily_people_vaccinated',      'first'),
            ('daily_people_vaccinated',       'last'),
            ('daily_people_vaccinated',       'mean'),
            ('daily_people_vaccinated',     'median'),
            ('daily_people_vaccinated',        'max'),
            (     'total_vaccinations',        'max'),
            (     'total_vaccinations', '<lambda_0>')],
           )

Each column currently has two names, so we need to user `droplevel` with `axis=1`

In [16]:
vacc_group = vacc_group.droplevel(0, axis=1) 
#vacc_group.columns = vacc_group.columns.droplevel(0)  #this is from older version of pandas
vacc_group

,,first,last,mean,median,max,max,<lambda_0>
0,Afghanistan,1367.0,9991.0,1.884408e+04,7585.0,188998.0,1.658658e+07,16.586584
1,Africa,0.0,343074.0,6.332570e+05,613776.5,1792254.0,7.995330e+08,799.533040
2,Albania,64.0,31.0,1.688170e+03,516.5,6816.0,3.070468e+06,3.070468
3,Algeria,30.0,0.0,1.345704e+04,9059.0,105248.0,1.526744e+07,15.267442
4,Andorra,66.0,0.0,7.545538e+01,1.0,854.0,1.569570e+05,0.156957
...,...,...,...,...,...,...,...,...
230,Wallis and Futuna,272.0,0.0,9.922504e+00,3.0,272.0,1.805800e+04,0.018058
231,World,0.0,10567.0,5.795624e+06,2647984.0,21071041.0,1.337684e+10,13376.837612
232,Yemen,4276.0,67.0,1.491194e+03,609.0,10240.0,1.258021e+06,1.258021
233,Zambia,106.0,3388.0,1.615831e+04,8387.0,47250.0,1.279211e+07,12.792112


Rename the columns

In [17]:
vacc_group.columns = ['location','daily_first','daily_last','daily_mean','daily_median','daily_max','total_max','total_max2']
vacc_group

,location,daily_first,daily_last,daily_mean,daily_median,daily_max,total_max,total_max2
0,Afghanistan,1367.0,9991.0,1.884408e+04,7585.0,188998.0,1.658658e+07,16.586584
1,Africa,0.0,343074.0,6.332570e+05,613776.5,1792254.0,7.995330e+08,799.533040
2,Albania,64.0,31.0,1.688170e+03,516.5,6816.0,3.070468e+06,3.070468
3,Algeria,30.0,0.0,1.345704e+04,9059.0,105248.0,1.526744e+07,15.267442
4,Andorra,66.0,0.0,7.545538e+01,1.0,854.0,1.569570e+05,0.156957
...,...,...,...,...,...,...,...,...
230,Wallis and Futuna,272.0,0.0,9.922504e+00,3.0,272.0,1.805800e+04,0.018058
231,World,0.0,10567.0,5.795624e+06,2647984.0,21071041.0,1.337684e+10,13376.837612
232,Yemen,4276.0,67.0,1.491194e+03,609.0,10240.0,1.258021e+06,1.258021
233,Zambia,106.0,3388.0,1.615831e+04,8387.0,47250.0,1.279211e+07,12.792112


`unstack` takes the innermost index and creates a column from it

In [18]:
vacc_df['year'] = pd.DatetimeIndex(vacc_df['date']).year

In [19]:
yr_mn_grp = vacc_df.groupby(['month','year'])[['daily_vaccinations']].mean().unstack()
yr_mn_grp 

daily_vaccinations                                            
year                2020           2021           2022          2023
month                                                               
1                    NaN  167026.954853  551628.826372  80208.691868
2                    NaN  203132.600303  415863.460103  43646.337672
3                    NaN  258197.645959  296595.597208  43313.990393
4                    NaN  352489.756990  201884.037037  27517.867797
5                    NaN  476568.340391  139863.826206           NaN
6                    NaN  678885.313223  155760.159879           NaN
7                    NaN  609030.806718  159514.789387           NaN
8                    NaN  674960.798302  142471.532533           NaN
9                    NaN  564751.448276  117420.280504           NaN
10                   NaN  454224.880803  107795.097293           NaN
11                   NaN  547136.443678   88009.096397           NaN
12          76743.146608  650254.655249  113621.403923           NaN

tidy up the table so it can be further used:

In [20]:
#yr_mn_grp.columns = yr_mn_grp.columns.droplevel(0) #older version
yr_mn_grp = yr_mn_grp.droplevel(0, axis=1) 
yr_mn_grp = yr_mn_grp.reset_index()
yr_mn_grp = yr_mn_grp.rename_axis(None, axis=1)
yr_mn_grp

,month,2020,2021,2022,2023
0,1,NaN,167026.954853,551628.826372,80208.691868
1,2,NaN,203132.600303,415863.460103,43646.337672
2,3,NaN,258197.645959,296595.597208,43313.990393
3,4,NaN,352489.756990,201884.037037,27517.867797
4,5,NaN,476568.340391,139863.826206,NaN
5,6,NaN,678885.313223,155760.159879,NaN
6,7,NaN,609030.806718,159514.789387,NaN
7,8,NaN,674960.798302,142471.532533,NaN
8,9,NaN,564751.448276,117420.280504,NaN
9,10,NaN,454224.880803,107795.097293,NaN


In [21]:
daily_grp = vacc_df.groupby(['year-month','location'])[['daily_vaccinations']].mean().unstack()
daily_grp = daily_grp.transpose()


### <span style="color:blue"> Exercise:</span>
>
> Remove the multi-index from `daily_grp`

In [22]:
#daily_grp.index

---
>A summary:
>
>* `groupby()` - group according to the columns specified
>
>* `reset_index()`  adds a numerical index
>
>* `pd.to_datetime(df['date'])` - changes the attribute type to datetime
>
>* `pd.DatetimeIndex(df['date']).month` - extracts the month from the datatime attribute
>
>* `apply` - applies a function on each row (axis =0) in the dataframe. Change to (axis = 1) to apply the function on each column [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html#pandas.DataFrame.apply)
>
>* `lambda` - small anonymous function
>
>* `agg` - apply multiple functions at once, one for each specified column [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html)
>
>* `unstack` - unstack the inner-most index onto a column
>
>* `droplevel(0, axis = 1)` - drops the highest (first) level in the column index of a multi-index dataframe
>
>* `transpose` - switch between columns and rows
---

#### This was a lot of information.

#### Keep your balance. Practice. You will make it.

<div>
<img src="https://raw.githubusercontent.com/nlihin/data-analytics/main/images/balance.jpg" width="500"/>
</div>

Photo by <a href="https://unsplash.com/@martinsanchez?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Martin Sanchez</a> on <a href="https://unsplash.com/s/photos/perfect-balance?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>
  